In [1]:
using LinearAlgebra, NBInclude

In [2]:
@nbinclude("(M)_Hofstadter_Real_Space_in_Julia.ipynb")

Nx=5, Ny=5

In [3]:
p=1;q=Ny #q değeri, ly'nin tam böleni olmak zorunda
alpha = p/q

0.2

In [4]:
# Single-Particle Hamiltonian with Twisted Angle (Bound-to-Bound Phases)
function HSP_T(Tx,Ty)
    H = zeros(Complex{Float64},Nx*Ny,Nx*Ny)
    for m in 1:Nx*Ny
        for n in 1:Nx*Ny
            if m in PerBC()[n] 
                
                if abs(xy[m,1]-xy[n,1])==Nx-1
                    if xy[m,1] > xy[n,1]
                        H[m,n] = -exp(-1im*2*pi*alpha*xy[m,2])*exp(-1im*Tx)
                    elseif xy[m,1] < xy[n,1]
                        H[m,n] = -exp(1im*2*pi*alpha*xy[m,2])*exp(1im*Tx)
                    end
                    
                elseif abs(xy[m,2]-xy[n,2])==Ny-1
                    if xy[m,2] > xy[n,2]
                        H[m,n] = -exp(1im*Ty)
                    elseif xy[m,2] < xy[n,2]
                        H[m,n] = -exp(-1im*Ty)
                    end
                    
                else
                    if xy[m,1] > xy[n,1]
                        H[m,n] = -exp(1im*2*pi*alpha*xy[m,2])
                    elseif xy[m,1] < xy[n,1]
                        H[m,n] = -exp(-1im*2*pi*alpha*xy[m,2])
                    else
                        H[m,n] = -exp(0)
                    end
                end
                
            else
                H[m,n] = 0
            end
            
        end
    end
    return H
end

HSP_T (generic function with 1 method)

In [5]:
HSP_T(0,0) == HSP(Nx,Ny,alpha) #check

true

In [6]:
E0, U0 = eigen(HSP_T(9,9))
# There is degeneracy which is needed 
i = sortperm(E0) #E'ler sıralı! Yani U'lar da sıralı demektir!
sorted = U0[:,i] #E'lere göre U'ları yeniden sıralayalım.
U0==sorted #U'lar zaten sıralıymış !

true

Pythonda yazılımış kodla kıyaslandığında E'leri aynı çıktığı halde U'ları aynı sırada gelmiyor?! Halbuki Julia'da ve Python'da da sıralı biçimde hesaplandı!

In [7]:
E0

25-element Vector{Float64}:
 -2.9052202568834344
 -2.9052202568834296
 -2.905220256883429
 -2.9052202568834287
 -2.905220256883423
 -1.3346819479735532
 -1.334681947973552
 -1.3346819479735474
 -1.3346819479735448
 -1.3346819479735397
  0.3380856139569663
  0.3380856139569676
  0.33808561395696995
  0.33808561395697145
  0.33808561395697306
  0.9380087150690739
  0.9380087150690756
  0.9380087150690769
  0.9380087150690785
  0.9380087150690791
  2.9638078758309363
  2.9638078758309367
  2.9638078758309367
  2.963807875830937
  2.9638078758309376

In [8]:
U0[:,2]

25-element Vector{ComplexF64}:
  -0.21619116964636123 - 0.07585442268631704im
   -0.3196518239737828 - 0.023770974423968964im
  -0.34705925934244874 + 0.026475465178481084im
    -0.187834330153973 + 0.14383532565055845im
   0.07427448481274852 + 0.12918517174403565im
  -0.21781071397609847 - 0.08295203938192464im
  0.008079069471999507 - 0.18184504291108022im
   0.26150294268044827 - 0.19308296781569156im
    0.3397098787967304 - 0.11720875713574454im
     0.302753471544226 - 0.08714843181829447im
  -0.11291266870850787 + 0.06207224902428557im
 -0.038906369913069457 + 0.07147028770286097im
  -0.09725675447838913 + 0.11575210453355174im
  -0.18254736239763944 - 0.06547670278082963im
  -0.03674091944847736 - 0.18164507153713955im
 -0.029935560327016178 + 0.011171373265538905im
    -0.098685926536161 - 0.05026831146630034im
 -0.041886898204377465 + 0.00524550477737961im
   0.03111954757079681 + 0.04668666696689681im
 -0.045877948914320725 - 0.002117700464645593im
   0.08370115488011448 - 

In [9]:
# Twisted Angle Space for Calculating Real Space Chern Numbers
Tsize=10
dx=2*pi/Tsize
dy=dx
Tx=collect(range(start=0, stop=2*pi, step=dx))
Ty=collect(range(start=0, stop=2*pi, step=dy))

11-element Vector{Float64}:
 0.0
 0.6283185307179586
 1.2566370614359172
 1.8849555921538759
 2.5132741228718345
 3.141592653589793
 3.7699111843077517
 4.39822971502571
 5.026548245743669
 5.654866776461628
 6.283185307179586

In [10]:
# Chern Calculation
Sum=0
n1=1;n2=6
for tx in range(start=1, stop=length(Tx))
    for ty in range(start=1, stop=length(Ty))
        w1, v1 = eigen(HSP_T(Tx[tx], Ty[ty]))
        v1 = v1[:,n1:n2]
        #------------------------------------
        w2, v2 = eigen(HSP_T(Tx[tx]+dx, Ty[ty]))
        v2 = v2[:,n1:n2]
        #------------------------------------
        w3, v3 = eigen(HSP_T(Tx[tx], Ty[ty]+dy))
        v3 = v3[:,n1:n2]
        #------------------------------------
        w4, v4 = eigen(HSP_T(Tx[tx]+dx, Ty[ty]+dy))
        v4 = v4[:,n1:n2]
        #----------LINK VARIABLES------------
        U1=det(*(adjoint(v1),v2))
        U1=U1/abs(U1)
        U2=det(*(adjoint(v2),v4))
        U2=U2/abs(U2)
        U3=det(*(adjoint(v3),v4))
        U3=U3/abs(U3)
        U4=det(*(adjoint(v1),v3))
        U4=U4/abs(U4)
        #----------BERRY CURVATURE-----------
        F=log(U1*U2*1/U3*1/U4)
        global Sum
        Sum=Sum+F
    end
end
Chern=1/(2*pi*1im)*Sum
println(Chern)

2.910525236838632 - 2.221368987253896e-16im
